In [1]:
import requests
from requests_html import HTMLSession

In [2]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'pragma': 'no-cache',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36',

}

def run(username):
    stars_url = "https://github.com/" + username + "?tab=stars"
    
    session = HTMLSession()
    resp = session.get(stars_url, headers=headers)
    
    # 获取类型
    types = resp.html.xpath('//ul[@class="filter-list"]/li/a/text()')
    types_result = [k.replace("  ","") for k in [j.replace("\n      ","") for j in [i.replace("\n        ","") for i in types]]]
    types_result = types_result[4:] # 去掉多余
    print(types_result)
    
    # 获取类型链接
    urls = resp.html.xpath('//ul[@class="filter-list"]/li/a/@href')
    urls_result = urls[4:] # 去掉多余
    print(urls_result)
    
    # 获取项目名称、评论、stars、fork
    # 判断是否有下一页，获取链接
    
def makeMarkdown():
    details_head_contents = "<details><summary>Contents</summary>"
    details_head_List = "<details><summary>List</summary>"
    datails_font = "</details>"
    
    # 模板制作：显示样式，是否需要stars&fork
    # 支持输入文件名，可有可无".md"
    
if __name__ == '__main__':
#     username = input("input your github username: ")
    username = "anlzou"
    data = run(username)

['C++', 'CSS', 'HTML', 'Java', 'JavaScript', 'Makefile', 'Python', 'Shell', 'TypeScript', 'Vue']
['https://github.com/anlzou?language=c%2B%2B&tab=stars', 'https://github.com/anlzou?language=css&tab=stars', 'https://github.com/anlzou?language=html&tab=stars', 'https://github.com/anlzou?language=java&tab=stars', 'https://github.com/anlzou?language=javascript&tab=stars', 'https://github.com/anlzou?language=makefile&tab=stars', 'https://github.com/anlzou?language=python&tab=stars', 'https://github.com/anlzou?language=shell&tab=stars', 'https://github.com/anlzou?language=typescript&tab=stars', 'https://github.com/anlzou?language=vue&tab=stars']
